In [24]:
#Libraries
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import textfeatures as tf
from nltk.tokenize import TweetTokenizer
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer, NaiveBayesAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## Obtaining Tweet Data

Tweet data gathered in the notebook for Tweet data queries are put into dataframes from their original JSON format. This will assist with adding COVID and Poll Data later for full analysis.

In [25]:
#All Covid tweets
All_Covid_tweets = pd.read_json('tweets/Covid_tweets5.json',lines=True)

#All Trump tweets
Trump_Covid_tweets = pd.read_json('tweets/Trump_Covid_tweets4.json', lines=True)

#All Cuomo tweets
Cuomo_Covid_tweets = pd.read_json('tweets/Cuomo_Covid_tweets5.json',lines=True)

#Baseline Tweets
NYTimes_tweets = pd.read_json('tweets/Nytimes_Covid_tweets5.json',lines=True)
#print( len(NYTimes_tweets))
WashingtonPost_tweets = pd.read_json('tweets/Washpost_tweets5.json',lines=True)
#print( len(Washpost_tweets3.json))

#combining NYTimes and Washington Post to get Baseline Tweets
Baseline_tweets = pd.concat([NYTimes_tweets,WashingtonPost_tweets],axis=0)

#Reformatting Date columns for later merge
All_Covid_tweets['date'] = pd.to_datetime(All_Covid_tweets['date'], format='%Y%m%d')
Trump_Covid_tweets['date'] = pd.to_datetime(Trump_Covid_tweets['date'], format='%Y%m%d')
Cuomo_Covid_tweets['date'] = pd.to_datetime(Cuomo_Covid_tweets['date'], format='%Y%m%d')
Baseline_tweets['date'] = pd.to_datetime(Baseline_tweets['date'], format='%Y%m%d')

In [26]:
#Combining all Tweet DFs into one
Master_Tweet_df = pd.concat([All_Covid_tweets,Trump_Covid_tweets,
                             Cuomo_Covid_tweets,Baseline_tweets],axis=0)

number_of_tweets = len(Master_Tweet_df)
print('The number of tweets in the dataset is:',number_of_tweets)

The number of tweets in the dataset is: 24681


## Obtaining Covid Data

COVID tracking data was pulled from [this repository](https://github.com/COVID19Tracking/covid-tracking-data/blob/master/data/us_daily.csv). I choose this resource because it is monitored daily and is maintained by a reputable organization - The Atlantic. While this data includes details around ICU populations and number of people on ventilators, I will be focused primarily on the number of positive cases and deathes in the United States.

In [27]:
# Covid data set
covid_data = pd.read_csv('https://raw.githubusercontent.com/COVID19Tracking/covid-tracking-data/master/data/us_daily.csv')

# formatting the date column to datetime
covid_data['date'] = pd.to_datetime(covid_data['date'], format='%Y%m%d')

In [28]:
covid_data = covid_data[['date','positive','death']]
covid_data.head()

,date,positive,death
0,2020-08-25,5749803,170353.0
1,2020-08-24,5713124,169206.0
2,2020-08-23,5678088,168863.0
3,2020-08-22,5639766,168291.0
4,2020-08-21,5593318,167262.0


### Adding Case/Death Data on Day of the Tweet

In [29]:
#merging the two datasets for all Covid data in one place
Master_Tweet_dataset = pd.merge(Master_Tweet_df,covid_data,on='date')
Master_Tweet_dataset.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,positive,death
0,1228397478954795009,1228397478954795008,2020-02-14 19:15:56,2020-02-14,14:15:56,EDT,143132365,nycemswatch,NYC EMS Watch,,...,,,,"[{'user_id': '143132365', 'username': 'NYCEMSw...",,,,,21,0.0
1,1228397478954795009,1228397478954795008,2020-02-14 19:15:56,2020-02-14,14:15:56,EDT,143132365,nycemswatch,NYC EMS Watch,,...,,,,"[{'user_id': '143132365', 'username': 'NYCEMSw...",,,,,21,0.0
2,1228397478954795009,1228397478954795008,2020-02-14 19:15:56,2020-02-14,14:15:56,EDT,143132365,nycemswatch,NYC EMS Watch,,...,,,,"[{'user_id': '143132365', 'username': 'NYCEMSw...",,,,,21,0.0
3,1231683951896383488,1231683951896383488,2020-02-23 20:55:12,2020-02-23,15:55:12,EDT,44984619,bobfoolery,Bob Cooper,,...,,,,"[{'user_id': '44984619', 'username': 'BobFoole...",,,,,122,0.0
4,1231683951896383488,1231683951896383488,2020-02-23 20:55:12,2020-02-23,15:55:12,EDT,44984619,bobfoolery,Bob Cooper,,...,,,,"[{'user_id': '44984619', 'username': 'BobFoole...",,,,,122,0.0


### Dropping Unneeded Columns

The following columns were deleted from the dataframe because they were mainly metadata that wouldn't be used in the various models that will be utilized later in this analysis.

In [30]:
#dropping unnecessary columns

Master_Tweet_dataset = Master_Tweet_dataset.drop(['cashtags', 'conversation_id', 'hashtags',
       'id','link', 'mentions', 'name', 'near', 'photos',
       'place', 'quote_url','reply_to', 'retweet',
       'retweet_date', 'retweet_id','source', 'time',
       'timezone', 'trans_dest', 'trans_src', 'translate','urls',
       'user_id', 'user_rt', 'user_rt_id',],axis=1)

### SCRUBBING OF TWEETS

To conduct most of the preprocessing of the tweet data, I utilized the Textfeatures library. This library assists with not only lowercase and remove punctuation and items such as hashtags, but provide useful data such as word count, average word length, stopwords count, and the character count of each tweet.

In [31]:
#pip install textfeatures
#Getting ride of duplicative column
Master_Tweet_dataset=Master_Tweet_dataset.drop('created_at',axis=1)
Master_Tweet_dataset.columns

Index(['date', 'username', 'tweet', 'replies_count', 'retweets_count',
       'likes_count', 'video', 'geo', 'positive', 'death'],
      dtype='object')

In [32]:
#Using textfeatures library text preprocessing
tf.word_count(Master_Tweet_dataset,"tweet",'word_count')
tf.avg_word_length(Master_Tweet_dataset,'tweet','avg_word_length')
tf.stopwords_count(Master_Tweet_dataset,'tweet','stopwords_count')
tf.char_count(Master_Tweet_dataset,'tweet','char_count')
tf.stopwords(Master_Tweet_dataset,'tweet','stopwords')
tf.clean(Master_Tweet_dataset,'tweet','clean_text')
Master_Tweet_dataset.head()

,date,username,tweet,replies_count,retweets_count,likes_count,video,geo,positive,death,word_count,avg_word_length,stopwords_count,char_count,stopwords,clean_text
0,2020-02-14,nycemswatch,Everything you need to know about AMR right he...,3,12,15,0,,21,0.0,37,9.780488,12,442,"[you, to, about, who, for, have, after, a, wit...",everything need know right emts work american ...
1,2020-02-14,nycemswatch,Everything you need to know about AMR right he...,3,12,15,0,,21,0.0,37,9.780488,12,442,"[you, to, about, who, for, have, after, a, wit...",everything need know right emts work american ...
2,2020-02-14,nycemswatch,Everything you need to know about AMR right he...,3,12,15,0,,21,0.0,37,9.780488,12,442,"[you, to, about, who, for, have, after, a, wit...",everything need know right emts work american ...
3,2020-02-23,bobfoolery,If/when COVID-19 becomes epidemic in the Unite...,0,0,0,0,,122,0.0,46,5.021739,15,276,"[in, the, will, down, as, has, been, in, and, ...",ifwhen covid becomes epidemic united states am...
4,2020-02-23,bobfoolery,If/when COVID-19 becomes epidemic in the Unite...,0,0,0,0,,122,0.0,46,5.021739,15,276,"[in, the, will, down, as, has, been, in, and, ...",ifwhen covid becomes epidemic united states am...


After making sure all of the text data is uniform and all special characters have been removed, I utiilized the TweetTokenizer library to tokenize every tweet to form the larger corpus. Tokenizing the tweets is an important step to assist the computer make sense of the text that was gathered through the tweets. Tokenization will allow for us to ascertain the topics being discussed in the tweets.

In [33]:
#Tokenizing the Corpus

clean_tweet = Master_Tweet_dataset['clean_text']
#Tweet Tokenizer 

ttknz = TweetTokenizer()

#creation of the corpus
#corpus = Master_Tweet_df['clean_tweets'].astype(str)
#corpus.dtypes

#tokenizing corpus
tok_corp = []
for sent in clean_tweet:
    toked = ttknz.tokenize(sent)
    tok_corp.append(toked)

In [34]:
#saving tok list or later use

with open('tok_corp_8_8.pickle','wb') as f:
    pickle.dump(tok_corp,f)

For Sentiment Analysis, the TextBlob library was utilized. The output of this analyis will give each tweet a score between -1 and 1. Scores closer to -1 can be classified as negative and scores around zero can be thought of as being neutral in sentiment. Finally, items with a score closer to 1 can be defined as having a positive sentiment.

In [35]:
#Sentiment Analysis - adding Sentiment rating to each tweet

#from twitter_nlp_toolkit.tweet_sentiment_classifier import tweet_sentiment_classifier

#tweets = Master_Tweet_df['clean_tweets']

tweets = Master_Tweet_dataset['clean_text']

Sentiment = []
for tweet in tweets:
    #Classifier = tweet_sentiment_classifier.SentimentAnalyzer()
    #sentiment = Classifier.predict_proba(tweet)
    blob = TextBlob(tweet,analyzer=PatternAnalyzer())
    rating = blob.sentiment.polarity
    Sentiment.append(rating)

Master_Tweet_dataset['Sentiment'] = Sentiment
#Master_Tweet_df['Sentiment'] = Master_Tweet_df['Sentiment'].astype(int)
#Master_Tweet_df['Sentiment'].round(decimals = 4)

#Master_Tweet_df['Sentiment'].head()

## Adding Poll Info

Polling data was collected from [RealClear Politics](https://www.realclearpolitics.com/epolls/2020/president/us/general_election_trump_vs_biden-6247.html). This source was chosen because they maintain a collection of polls for the US Presidential election and they do a good job of showing the overall momentum a candidate may have at a given time. To assist with the merge, I manipulated the date column in excel so that the merge would be successful as attempts to make the changes with Pandas were unsuccessful.

In [36]:
#Uploading Poll data
poll_data = pd.read_csv('data/poll_data_dates.csv')
poll_data = poll_data.dropna()

#Converting Date columns to integer so merge will work
Master_Tweet_dataset['date'] = pd.to_datetime(Master_Tweet_dataset['date'])
Master_Tweet_dataset['date'] = Master_Tweet_dataset['date'].astype(int)
poll_data['date'] = pd.to_datetime(poll_data['date'])
poll_data['date'] = poll_data['date'].astype(int)


In [37]:
#Merging with Poll Data

#poll_data = pd.read_csv('data/poll_data_dates.csv')
#pd.to_datetime(poll_data['Date']) #converting to datetime object for merge purposes
#pd.to_datetime(Master_Tweet_df['Date']) #converting to datetime object for merge purposes

left = Master_Tweet_dataset.sort_values(by='date')
right = poll_data.sort_values(by='date')

Master_Tweet_dataset = pd.merge_asof(left,right,on='date',allow_exact_matches=False)

## Creation of Target Column

For this analysis, determining the public's perception of the Covid response in America was critical. To learn this based on tweets, Latent Dirichlet Allocation (LDA) was employed. LDA is an unsupervised machine-learning model that takes documents as input and finds topics as output. Further,as detailed on [Towards Data Science](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0), LDA is a generative probabilistic model that assumes each topic is a mixture over an underlying set of words, and each document is a mixture of over a set of topic probabilities.

To generate these topics, count vectorization was used to convert the corpus of documents into a matrix of token counts which was later fed into the LDA model so that topics could be generated. As you will see, the ten topics will have the most common words for that topic and the associated percentage that word appears in that topic. 

In [38]:
#Assiging topics to each tweet  -code inspired by stackabuse


#Vectorizing docs
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(clean_tweet)

#fitting LDA Model

LDA = LatentDirichletAllocation(n_components=10, random_state=42)
LDA_model_8_22 = LDA.fit(doc_term_matrix)

#transforming to get topic numbers
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

#creating column of Topics
Master_Tweet_dataset['Topic'] = topic_values.argmax(axis=1)

In [39]:
# top words for each topic
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])

Top 10 words for topic #0:
['blood', 'government', 'make', 'virus', 'people', 'trump', 'coronavirus', 'american', 'covid', 'pandemic']
Top 10 words for topic #1:
['thousands', 'trumps', 'people', 'death', 'deaths', 'coronavirus', 'trump', 'pandemic', 'covid', 'american']
Top 10 words for topic #2:
['deaths', 'people', 'trumpdeathclock', 'delayed', 'cost', 'coronavirus', 'trumps', 'lives', 'american', 'covid']
Top 10 words for topic #3:
['compared', 'disease', 'americans', 'people', 'countries', 'health', 'coronavirus', 'pandemic', 'american', 'covid']
Top 10 words for topic #4:
['customers', 'gave', 'service', 'especially', 'provide', 'family', 'relief', 'america', 'american', 'covid']
Top 10 words for topic #5:
['black', 'free', 'time', 'social', 'fund', 'support', 'nurses', 'pandemic', 'american', 'covid']
Top 10 words for topic #6:
['delta', 'coronavirus', 'lines', 'including', 'pandemic', 'united', 'states', 'airlines', 'american', 'covid']
Top 10 words for topic #7:
['crisis', 'em

In [40]:
#Dictionary to create Target Column
category_dict = {}
for key in [0,4,5,8,1,3]:
    category_dict[key] = 'Bad Response'
for key in [10,6,2,7,9]:
    category_dict[key] = 'Good Response'
    
#Creation of Target Column
Master_Tweet_dataset['Target'] = Master_Tweet_dataset['Topic'].map(category_dict)

## Model Ready DataFrame

To ensure that the data fed to our models is uniform, this last step just dropped columns that were either superflous or contained text that our number-driven models cannot make any use of. For portability, the final dataframe was pickled so that it could be recalled in the Models notebook.

In [41]:
Master_Tweet_dataset = Master_Tweet_dataset.drop(['date','username','video','geo','clean_text','Start Date', 
                                                  'End Date','tweet','Sample','replies_count', 'retweets_count',
                                                  'MoE','stopwords','Poll','MoE'],axis=1)

In [42]:
Master_Tweet_dataset['death'] = Master_Tweet_dataset['death'].fillna(0)
Master_Tweet_dataset.head()

,likes_count,positive,death,word_count,avg_word_length,stopwords_count,char_count,Sentiment,Biden (D),Trump (R),Spread,Topic,Target
0,909,6,0.0,34,5.484848,12,215,0.000000,50.0,44.0,Biden +6,2,Good Response
1,909,6,0.0,34,5.484848,12,215,0.000000,50.0,44.0,Biden +6,2,Good Response
2,79,7,0.0,16,7.200000,2,124,0.136364,50.0,44.0,Biden +6,2,Good Response
3,79,7,0.0,16,7.200000,2,124,0.136364,50.0,44.0,Biden +6,7,Good Response
4,15,21,0.0,37,9.780488,12,442,0.071429,50.0,43.0,Biden +7,7,Good Response


In [43]:
pickle.dump(Master_Tweet_dataset,open('Master_Tweet_dataset.pickle','wb'))

In [44]:
len(Master_Tweet_dataset['Target'])

24681